In [1]:
import pandas as pd
import numpy as np

/anaconda3/envs/python36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/envs/python36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
df = pd.read_csv('combined_stats.csv')
#df['TEAM'] = df["TEAM"].str.lower()
#df.to_csv('combined_stats.csv',encoding='utf-8')



In [6]:
df = pd.read_csv('combined_stats.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,TEAM,GP,PPG,FGM-game,FGA-game,FGM-total,FGA-total,FG%,...,2P%,PPS,FG%.adj,team,games_played,assist,assist_per_game,turnover,turnover_per_game,assist/to
0,0,0,gonzaga,33,88.8,32.2,60.5,1062,1997,0.532,...,0.623,1.47,0.596,Gonzaga,33,601,18.2,342,10.4,1.76
1,1,1,south dakota state,32,84.9,29.9,59.4,957,1901,0.503,...,0.568,1.43,0.587,South Dakota State,32,476,14.9,379,11.8,1.26
2,2,2,dayton,32,72.9,27.3,54.2,872,1734,0.503,...,0.592,1.35,0.561,Dayton,32,529,16.5,403,12.6,1.31
3,3,3,belmont,31,87.4,31.8,63.7,987,1976,0.499,...,0.599,1.37,0.582,Belmont,31,616,19.9,360,11.6,1.71
4,4,4,georgia southern,33,82.6,30.7,61.5,1012,2028,0.499,...,0.579,1.34,0.549,Georgia Southern,33,403,12.2,452,13.7,0.89


In [7]:
df.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'TEAM', 'GP', 'PPG', 'FGM-game',
       'FGA-game', 'FGM-total', 'FGA-total', 'FG%', '2PM', '2PA', '2P%', 'PPS',
       'FG%.adj', 'team', 'games_played', 'assist', 'assist_per_game',
       'turnover', 'turnover_per_game', 'assist/to'],
      dtype='object')

In [8]:
def school_list():
    list_schools = df['TEAM'].values.tolist()
    return(list_schools)

In [9]:
test = school_list()
test

['gonzaga',
 'south dakota state',
 'dayton',
 'belmont',
 'georgia southern',
 'tennessee',
 'montana',
 'murray state',
 'yale',
 'loyola-chicago',
 'wofford',
 'liberty',
 'michigan state',
 'gardner-webb',
 'hofstra',
 'east tennessee state',
 'utah valley',
 'little rock',
 'charleston',
 'lipscomb',
 'northeastern',
 'lehigh',
 'creighton',
 'northern kentucky',
 'purdue fort wayne',
 'boston university',
 'colgate',
 'kentucky',
 'virginia',
 'fairleigh dickinson',
 'duke',
 'iowa state',
 'colorado state',
 'furman',
 'weber state',
 'virginia tech',
 'omaha',
 'william & mary',
 'american',
 'abilene christian',
 'utah state',
 "saint mary's",
 'umass lowell',
 'austin peay',
 'samford',
 'mississippi state',
 'richmond',
 'texas tech',
 'northern illinois',
 'drake',
 'ball state',
 'boise state',
 'xavier',
 'byu',
 'southern miss',
 'san francisco',
 'depaul',
 'southern illinois',
 'incarnate word',
 'georgia state',
 'nevada',
 'oakland',
 'north dakota',
 'utah',
 'north

In [10]:


new_df = df[['FGA-game','assist_per_game','turnover_per_game']]
new_df.head()

,FGA-game,assist_per_game,turnover_per_game
0,60.5,18.2,10.4
1,59.4,14.9,11.8
2,54.2,16.5,12.6
3,63.7,19.9,11.6
4,61.5,12.2,13.7


In [11]:
def get_team(team):
    
    team_stat= df.loc[df['TEAM'] == team]
    new_df = team_stat[['FGA-game','assist_per_game','turnover_per_game']].values.tolist()
    stats = new_df[0]
    #print(stats)
    return(stats)

In [13]:
get_team('auburn')

[60.9, 14.4, 12.3]

In [14]:
def model_prep(home,away):
    home_stats = get_team(home)
    away_stats = get_team(away)
    model_list = []
    
    for x in range(0,3):
        model_list.append(home_stats[x])
        model_list.append(away_stats[x])
    model_stats= {'h_field_goals_att':model_list[0],'a_field_goals_att':model_list[1], 'h_assists':model_list[2],'a_assists':model_list[3],
                  'h_turnovers':model_list[4],'a_turnovers':model_list[5]}
    model_df = pd.DataFrame(model_stats, index=[0])
    return(model_df)
    #return(model_list)

In [15]:
model_prep("gonzaga","dayton")

,h_field_goals_att,a_field_goals_att,h_assists,a_assists,h_turnovers,a_turnovers
0,60.5,54.2,18.2,16.5,10.4,12.6


In [16]:
from keras.models import load_model
ncaa_model = load_model("deep_neural_ncaa_trained.h5")

Using TensorFlow backend.


In [12]:
from sklearn.externals import joblib

scaler = joblib.load("xscaler.save") 

In [41]:
test = scaler.transform(model_prep("Gonzaga","Auburn"))
test
answer = ncaa_model.predict(test)
answer
result = np.argmax(answer,axis=1,out=None)
result

array([1])

In [44]:
def model_predict(home,away):
    
    team_stats= model_prep(home,away)
    x_valid = scaler.transform(team_stats)
    prediction= ncaa_model.predict(x_valid)
    #prediction= ncaa_model.predict(team_stats)
    result = np.argmax(prediction,axis=None,out=None)
    print(result)
    if result == 0 :
        return(f'{home} wins{result}')
    else:
        return(f'{away} wins{result}')
    

In [46]:
model_predict("Gonzaga","Auburn")

1


'Auburn wins1'